# Getting Started

## Tensors

Tensors are similar to NumPy’s ndarrays, with the addition being that Tensors can also be used on a GPU to accelerate computing.

In [1]:
import torch

> An uninitialized matrix is declared, but does not contain definite known values before it is used. When an uninitialized matrix is created, whatever values were in the allocated memory at the time will appear as the initial values.

In [6]:
x = torch.empty(5, 3)  # uninitialized
print(x)

x = torch.rand(5, 3)  # initialized
print(x)

x = torch.zeros(5, 3, dtype=torch.long)
print(x)

x = torch.tensor([5.5, 3])
print(x)

tensor([[0.5470, 0.0137, 0.2217],
        [0.0604, 0.6369, 0.4298],
        [0.2662, 0.3210, 0.6745],
        [0.5556, 0.8333, 0.9057],
        [0.6497, 0.5925, 0.1911]])
tensor([[0.4398, 0.7884, 0.3267],
        [0.5537, 0.8500, 0.7284],
        [0.5717, 0.0264, 0.4086],
        [0.8925, 0.5700, 0.2915],
        [0.4727, 0.7799, 0.8360]])
tensor([[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]])
tensor([5.5000, 3.0000])


or create a tensor based on an existing tensor. These methods will reuse properties of the input tensor, e.g. dtype, unless new values are provided by user

In [7]:
x = x.new_ones(5, 3, dtype=torch.double)      # new_* methods take in sizes
print(x)

x = torch.randn_like(x, dtype=torch.float)    # override dtype!
print(x)                                      # result has the same size

tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]], dtype=torch.float64)
tensor([[ 0.0191, -0.3859,  0.3316],
        [-0.0963,  0.2201, -1.1670],
        [ 0.1447, -0.5729, -0.8914],
        [ 1.6304,  0.6642, -0.6040],
        [-0.6486,  1.0853, -0.5379]])


In [8]:
print(x.size())

torch.Size([5, 3])


> torch.Size is in fact a tuple, so it supports all tuple operations.

## Operations

There are multiple syntaxes for operations. In the following example, we will take a look at the addition operation.

In [9]:
y = torch.rand(5, 3)
print(x + y)

tensor([[ 0.6201,  0.1937,  0.3417],
        [ 0.6228,  0.9402, -0.7783],
        [ 1.0853, -0.4898, -0.0204],
        [ 2.5409,  0.7702,  0.3797],
        [-0.2227,  1.9154, -0.4290]])


In [12]:
print(torch.add(x, y))

result = torch.empty(5, 3)
torch.add(x, y, out=result)
print(result)

tensor([[ 0.6392, -0.1922,  0.6733],
        [ 0.5265,  1.1603, -1.9453],
        [ 1.2299, -1.0627, -0.9118],
        [ 4.1713,  1.4345, -0.2243],
        [-0.8714,  3.0007, -0.9669]])
tensor([[ 0.6392, -0.1922,  0.6733],
        [ 0.5265,  1.1603, -1.9453],
        [ 1.2299, -1.0627, -0.9118],
        [ 4.1713,  1.4345, -0.2243],
        [-0.8714,  3.0007, -0.9669]])


Addition: in place

> Any operation that mutates a tensor in-place is post-fixed with an _. For example: x.copy_(y), x.t_(), will change x.

In [13]:
y.add_(x)
print(y)

tensor([[ 0.6392, -0.1922,  0.6733],
        [ 0.5265,  1.1603, -1.9453],
        [ 1.2299, -1.0627, -0.9118],
        [ 4.1713,  1.4345, -0.2243],
        [-0.8714,  3.0007, -0.9669]])


Resizing: If you want to resize/reshape tensor, you can use torch.view:

In [14]:
x = torch.randn(4, 4)  # randn 标准正态分布
y = x.view(16)
z = x.view(-1, 8)  # the size -1 is inferred from other dimensions，从其他维度推理得出
print(x.size(), y.size(), z.size())

torch.Size([4, 4]) torch.Size([16]) torch.Size([2, 8])


If you have a one element tensor, use .item() to get the value as a Python number

In [17]:
x = torch.randn(1)
print(x)
print(x.item())

tensor([1.2346])
1.2346292734146118


# NumPy Bridge

Converting a Torch Tensor to a NumPy array and vice versa is a breeze.

The Torch Tensor and NumPy array will share their underlying memory locations (if the Torch Tensor is on CPU), and changing one will change the other.

## Converting a Torch Tensor to a NumPy Array

In [23]:
a = torch.ones(5)
print(a)

b = a.numpy()
print(b)

tensor([1., 1., 1., 1., 1.])
[1. 1. 1. 1. 1.]


See how the numpy array changed in value.

In [24]:
a.add_(1)  # a.add_(torch.ones(5)) 两种表达式等价
print(a)
print(b)

tensor([2., 2., 2., 2., 2.])
[2. 2. 2. 2. 2.]


Converting NumPy Array to Torch Tensor

In [25]:
import numpy as np
a = np.ones(5)
b = torch.from_numpy(a)
np.add(a, 1, out=a)
print(a)
print(b)

[2. 2. 2. 2. 2.]
tensor([2., 2., 2., 2., 2.], dtype=torch.float64)


All the Tensors on the CPU except a CharTensor support converting to NumPy and back.

CPU上除了CharTensor以外的所有张量都支持NumPy和back的转换。

# CUDA Tensors

Tensors can be moved onto any device using the .to method.

In [29]:
# let us run this cell only if CUDA is available
# We will use ``torch.device`` objects to move tensors in and out of GPU

x = torch.rand(10000, 10000)
if torch.cuda.is_available():
    device = torch.device("cuda")          # a CUDA device object
    y = torch.ones_like(x, device=device)  # directly create a tensor on GPU
    x = x.to(device)  # or just use strings ``.to("cuda")``
    z = x + y
    print(z)
    print(z.to("cpu", torch.double))  # ``.to`` can also change dtype together!

tensor([[1.9496, 1.6639, 1.4125,  ..., 1.6670, 1.7353, 1.4324],
        [1.5201, 1.1864, 1.4011,  ..., 1.1834, 1.9040, 1.8508],
        [1.2237, 1.4879, 1.2545,  ..., 1.2233, 1.2993, 1.4883],
        ...,
        [1.6296, 1.1912, 1.4555,  ..., 1.8398, 1.5661, 1.1045],
        [1.6690, 1.2420, 1.2312,  ..., 1.2655, 1.0327, 1.5003],
        [1.0543, 1.7201, 1.2721,  ..., 1.2174, 1.8885, 1.4345]],
       device='cuda:0')
tensor([[1.9496, 1.6639, 1.4125,  ..., 1.6670, 1.7353, 1.4324],
        [1.5201, 1.1864, 1.4011,  ..., 1.1834, 1.9040, 1.8508],
        [1.2237, 1.4879, 1.2545,  ..., 1.2233, 1.2993, 1.4883],
        ...,
        [1.6296, 1.1912, 1.4555,  ..., 1.8398, 1.5661, 1.1045],
        [1.6690, 1.2420, 1.2312,  ..., 1.2655, 1.0327, 1.5003],
        [1.0543, 1.7201, 1.2721,  ..., 1.2174, 1.8885, 1.4345]],
       dtype=torch.float64)
